# 1、后量化训练
## 1.1 线性量化
https://zhuanlan.zhihu.com/p/156835141

scale:
> s = rmax - rmin / qmax - qmin

zero point:
> z = round(qmax - rmax / s)

### 1. 量化基本公式

In [1]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function

In [2]:
# 计算 scale 和 zero_point 的基本公式
def calcu_scale_and_zeropoint(min_val, max_val, num_bits=8):
    q_min = 0.
    q_max = 2. ** num_bits - 1
    scale = float((max_val - min_val) / (q_max - q_min))
    zero_point = np.clip(int(q_max - max_val / scale), q_min, q_max)
    
    return scale, zero_point

In [3]:
# tensor 量化和反量化
def quantize_tensor(x, scale, zero_point, num_bits=8, signed=False):
    if signed:
        q_min = - 2. ** (num_bits - 1)
        q_max = 2. ** (num_bits - 1) - 1.
    else:
        q_min = 0.
        q_max = 2. ** num_bits -1.
        
    q_x = x / scale + zero_point
    q_x.clamp_(q_min, q_max).round()  # q=round(r/S+Z)
    
    return q_x.float()   # 由于pytorch不支持int类型的运算，因此我们还是用float来表示整数

def dequantize_tensor(q_x, scale, zero_point):
    return scale * (q_x -zero_point)  # r=S(q-Z)

### 2. 封装成类

In [4]:
class QParam:
    def __init__(self, num_bits=8):
        self.num_bits = num_bits
        self.scale = None
        self.zero_point = None
        self.min = None
        self.max = None
    
    def update(self, tensor):
        self.max = max(0, tensor.max()) if self.max is None \
                                        else max(0, self.max, tensor.max())
        self.min = min(0, tensor.min()) if self.min is None \
                                        else min(0, self.min, tensor.min())
        self.scale, self.zero_point = calcu_scale_and_zeropoint(self.min, self.max, self.num_bits)
        
    def quantize_tensor(self, tensor):
        return quantize_tensor(tensor, self.scale, self.zero_point, num_bits=self.num_bits)
    
    def dequantize_tensor(self, q_x):
        return dequantize_tensor(q_x, self.scale, self.zero_point)        

### 3、量化网络基类定义

In [5]:
class QModule(nn.Module):
    def __init__(self, has_qin=True, has_qout=True, num_bits=8):
        # 指定量化的位数外，还需指定是否提供量化输入 (qin) 及输出参数 (qout)
        # 不是每一个网络模块都需要统计输入的 min、max，大部分中间层都是用上一层的 qout 来作为自己的 qin 的，
        # 另外有些中间层的激活函数也是直接用上一层的 qin 来作为自己的 qin 和 qout。
        super(QModule, self).__init__()
        if has_qin:
            self.q_in = QParam(num_bits)
        if has_qout:
            self.q_out = QParam(num_bits)
    
    def freeze(self):
        # 函数会在统计完 min、max 后发挥作用
        # 很多项是可以提前计算好的，freeze 就是把这些项提前固定下来
        # 同时也将网络的权重由浮点实数转化为定点整数。
        pass
    
    def quantize_inference(self, x):
        # 在量化 inference 的时候会使用
        raise NotImplementedError('quantize_inference should be implemented.')

### 4、伪量化定义

In [6]:
# 前向推导中，先量化再反量化，引入噪声
class FakeQuantize(Function):

    @staticmethod
    def forward(ctx, x, qparam):
        x = qparam.quantize_tensor(x)
        x = qparam.dequantize_tensor(x)
        return x

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output, None

fake_quantize = FakeQuantize.apply

### 4、量化卷积层类的定义
<img src="https://www.zhihu.com/equation?tex=a%3D%5Csum_%7Bi%7D%5EN+w_i+x_i%2Bb+%5Ctag%7B1%7D+" alt="[公式]" style="zoom:80%;" />
由此得到量化的公式
<img src="https://www.zhihu.com/equation?tex=S_a+%28q_a-Z_a%29%3D%5Csum_%7Bi%7D%5EN+S_w%28q_w-Z_w%29S_x%28q_x-Z_x%29%2BS_b%28q_b-Z_b%29+%5Ctag%7B2%7D+" alt="[公式]" style="zoom:80%;" />
<img src="https://www.zhihu.com/equation?tex=q_a%3D%5Cfrac%7BS_w+S_x%7D%7BS_a%7D%5Csum_%7Bi%7D%5EN+%28q_w-Z_w%29%28q_x-Z_x%29%2B%5Cfrac%7BS_b%7D%7BS_a%7D%28q_b-Z_b%29%2BZ_a+%5Ctag%7B3%7D+" alt="[公式]" style="zoom:80%;" />


> <img src="./image/image.png" style="zoom:60%;" />

经过调整：

<img src="https://www.zhihu.com/equation?tex=%5Cbegin%7Balign%7D+q_a%26%3D%5Cfrac%7BS_w+S_x%7D%7BS_a%7D%28%5Csum_%7Bi%7D%5EN%28q_w-Z_w%29%28q_x-Z_x%29%2Bq_b%29%2BZ_a+%5Cnotag+%5C%5C+%26%3DM%28%5Csum_%7Bi%7D%5EN+q_wq_x-%5Csum_i%5EN+q_wZ_x-%5Csum_i%5EN+q_xZ_w%2B%5Csum_i%5ENZ_wZ_x%2Bq_b%29%2BZ_a+%5Ctag%7B4%7D+%5Cend%7Balign%7D+" alt="[公式]" style="zoom:80%;" />

In [7]:
class QConv2d(QModule):
    def __init__(self, conv_module, has_qin=True, has_qout=True, num_bits=8):
        super(QConv2d, self).__init__(has_qin, has_qout, num_bits)
        self.num_bits = num_bits
        self.conv_module = conv_module
        self.q_weight = QParam(num_bits=num_bits)
        
    def forward(self, x):
        if hasattr(self, 'q_in'):
            # 更新量化输入 q_in 的参数
            self.q_in.update(x)
            # 利用量化输入 q_in 伪量化全精度输入 x，引入噪声
            x = fake_quantize(x, self.q_in)
            
        self.q_weight.update(self.conv_module.weight.data)
        x = F.conv2d(input = x, 
                     weight = fake_quantize(self.conv_module.weight, self.q_weight), 
                     bias = self.conv_module.bias, 
                     stride = self.conv_module.stride,
                     padding = self.conv_module.padding, 
                     dilation=self.conv_module.dilation, 
                     groups = self.conv_module.groups)
        
        if hasattr(self, 'q_out'):
            self.q_out.update(x)
            x = fake_quantize(x, self.q_out)
            
        return x
    
    def freeze(self, q_in=None, q_out=None):
        if hasattr(self, 'q_in') and q_in is not None:
            raise ValueError('q_in has been provided in init function.')
        if not hasattr(self, 'q_in') and q_in is None:
            raise ValueError('q_in is not existed, should be provided.')

        if hasattr(self, 'q_out') and q_out is not None:
            raise ValueError('q_out has been provided in init function.')
        if not hasattr(self, 'q_out') and q_out is None:
            raise ValueError('q_out is not existed, should be provided.')
            
        if q_in is not None:
            self.q_in = q_in
        if q_out is not None:
            self.q_out = q_out
        
        # 计算 M = s_w * s_in / s_out
        self.M = self.q_weight.scale * self.q_in.scale / self.q_out.scale
        
        # 量化卷积层中的权重
        self.conv_module.weight.data = self.q_weight.quantize_tensor(self.conv_module.weight.data) \
                                        - self.q_weight.zero_point
        # 量化卷积层中的偏置
        self.conv_module.bias.data = quantize_tensor(self.conv_module.bias.data, scale=self.q_in.scale * self.q_weight.scale,
                                                     zero_point=0, num_bits=32, signed=True)
        
    
    def quantize_inference(self, x):
        x = x - self.q_in.zero_point
        x = self.conv_module(x)
        x = self.M * x
        x.round_()
        x = x + self.q_out.zero_point
        x.clamp_(0., 2. ** self.num_bits - 1.).round_()
        return x 

### 5、量化线性层类的定义

In [8]:
class QLinear(QModule):
    def __init__(self, fc_module, has_qin=True, has_qout=True, num_bits=8):
        super(QLinear, self).__init__(has_qin, has_qout, num_bits)
        self.num_bits = num_bits
        self.fc_module = fc_module
        self.q_weight = QParam(num_bits=num_bits)
        
    def forward(self, x):
        if hasattr(self, 'q_in'):
            self.q_in.update(x)
            x = fake_quantize(x, self.q_in)
            
        self.q_weight.update(self.fc_module.weight.data)
        x = F.linear(input = x, 
                     weight = fake_quantize(self.fc_module.weight, self.q_weight), 
                     bias = self.fc_module.bias)
        
        if hasattr(self, 'q_out'):
            self.q_out.update(x)
            x = fake_quantize(x, self.q_out)
            
        return x
    
    def freeze(self, q_in=None, q_out=None):
        if hasattr(self, 'q_in') and q_in is not None:
            raise ValueError('q_in has been provided in init function.')
        if not hasattr(self, 'q_in') and q_in is None:
            raise ValueError('q_in is not existed, should be provided.')

        if hasattr(self, 'q_out') and q_out is not None:
            raise ValueError('q_out has been provided in init function.')
        if not hasattr(self, 'q_out') and q_out is None:
            raise ValueError('q_out is not existed, should be provided.')
            
        if q_in is not None:
            self.q_in = q_in
        if q_out is not None:
            self.q_out = q_out
        
        # 计算 M = s_w * s_in / s_out
        self.M = self.q_weight.scale * self.q_in.scale / self.q_out.scale
        
        # 量化卷积层中的权重
        self.fc_module.weight.data = self.q_weight.quantize_tensor(self.fc_module.weight.data) \
                                        - self.q_weight.zero_point
        # 量化卷积层中的偏置
        self.fc_module.bias.data = quantize_tensor(self.fc_module.bias.data, scale=self.q_in.scale * self.q_weight.scale,
                                                     zero_point=0, num_bits=32, signed=True)
        
    def quantize_inference(self, x):
        x = x - self.q_in.zero_point
        x = self.fc_module(x)
        x = self.M * x
        x.round_()
        x = x + self.q_out.zero_point
        x.clamp_(0., 2. ** self.num_bits - 1.).round_()
        return x 

### 6、量化ReLu 类的定义

In [9]:
class QReLU(QModule):
    def __init__(self, has_qin=False, num_bits=None):
        super(QReLU, self).__init__(has_qin=has_qin,num_bits=num_bits)
        
    def forward(self, x):
        if hasattr(self, 'q_in'):
            self.q_in.update(x)
            x = fake_quantize(x, self.q_in)
            
        x = F.relu(x)
        
        return x
        
    def freeze(self, q_in=None):
        if hasattr(self, 'q_in') and q_in is not None:
            raise ValueError('q_in has been provided in init function.')
        if not hasattr(self, 'q_in') and q_in is None:
            raise ValueError('q_in is not existed, should be provided.')
            
        if q_in is not None:
            self.q_in = q_in
    
    def quantize_inference(self, x):
        x = x.clone()
        x[x < self.q_in.zero_point] = self.q_in.zero_point
        return x

### 7、量化最大池化层类的定义

In [10]:
class QMaxPooling2d(QModule):
    def __init__(self, kernel_size=3, stride=1, padding=0, has_qin=False, num_bits=None):
        super(QMaxPooling2d, self).__init__(has_qin=has_qin, num_bits=num_bits)
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
    
    def forward(self, x):
        if hasattr(self, 'q_in'):
            self.q_in.update(x)
            x = fake_quantize(x, self.q_in)
            
        x = F.max_pool2d(x, self.kernel_size, self.stride, self.padding)
        
        return x
        
    def freeze(self, q_in=None):
        if hasattr(self, 'q_in') and q_in is not None:
            raise ValueError('q_in has been provided in init function.')
        if not hasattr(self, 'q_in') and q_in is None:
            raise ValueError('q_in is not existed, should be provided.')
            
        if q_in is not None:
            self.q_in = q_in
    
    def quantize_inference(self, x):
        return F.max_pool2d(x, self.kernel_size, self.stride, self.padding)

### 8、卷积BNReLU合并

In [11]:
# class QConvBNReLU(QModule):
#     def __init__(self, conv_module, bn_module, has_qin=True, has_qout=True, num_bits=8):
#         super(QConvBNReLU, self).__init__(has_qin, has_qout, num_bits)
#         self.num_bits = num_bits
#         self.conv_module = conv_module
#         self.bn_module = bn_module
#         self.q_weight = QParam(num_bits=num_bits)
#         self.q_bias = QParam(num_bits=32)
        
#     def forward(self, x):
#         if hasattr(self, 'q_in'):
#             self.q_in.update(x)
#             x = fake_quantize(x, self.q_in)
            
#         if self.training:
            